<a href="https://colab.research.google.com/github/Guo-bot-1998/Appendicitis/blob/master/Appendicitis_colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import

In [ ]:
!pip install timm
!pip install tqdm
!pip install kora
!pip install fake_useragent

In [ ]:
import nibabel as nib
import numpy as np
import os
import pandas as pd
import nibabel as nib
import torch
import torch.nn as nn
import torch.optim as optim
import random
import re
import json
import timm
import gdown
from torch.utils.data import Dataset, DataLoader
from kora.xattr import get_id
import asyncio
import aiohttp
import nest_asyncio
from tqdm.asyncio import tqdm
nest_asyncio.apply()
loop = asyncio.get_event_loop()

## 存取

In [ ]:
def get_data_disk(driveroot, targetdir):
  """從雲端資料夾targetdir下載檔案到colab root底下同名資料夾
  只下載沒有下載的檔案.

  driveroot為雲端檔案資料夾root"""

  if not os.path.exists('/content/' + targetdir):
    os.makedirs(targetdir)
    print(f"making {targetdir} dir...")

  files_on_drive = os.listdir(driveroot+'/'+targetdir)
  files_in_localdir = os.listdir(f"/content/{targetdir}")
  files_to_download = [f for f in files_on_drive if f not in files_in_localdir]
  files = [(filename, get_id(f'{driveroot}/{targetdir}/{filename}')) for filename in files_to_download]


  for idx, (name, file_id) in enumerate(files):

    if not file_id:
      print(f"未找到文件: {name}")
      continue

    # 下载文件
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while not done:
      status, done = downloader.next_chunk()
      # print(f"下载 {name} 进度: {int(status.progress() * 100)}%")

    # 将文件内容写入本地文件
    local_file_path = os.path.join('/content', targetdir, name)

    with open(local_file_path, 'wb') as f:
      fh.seek(0)
      f.write(fh.read())

    print(f"{idx+1}/{len(files)}: {name} downloaded")




async def download_file(session, name, file_id, file_path, token):
  if not file_id:
    print(f"未找到文件: {name}")
    return

  download_url = f"https://www.googleapis.com/drive/v3/files/{file_id}?alt=media"
  headers = {"Authorization": f"Bearer {token}"}

  async with session.get(download_url, headers=headers) as response:
    if response.status == 200:
      with open(file_path, 'wb') as f:
        while True:
          chunk = await response.content.read(1024)
          if not chunk:
            break
          f.write(chunk)
      # print(f"Downloaded: {name}...")
    else:
      print(f"Failed: {name}!! status: {response.status}")


# 定义异步获取数据的主函数
async def get_data_disk_async(driveroot, targetdir, token):
  # 生成文件夹的绝对路径
  drive_folder_path = os.path.join(driveroot, targetdir)
  local_folder_path = os.path.join('/content', targetdir)

  if not os.path.exists(local_folder_path):
    os.makedirs(local_folder_path)
    print(f"making {targetdir} dir...")

  # 获取文件列表
  files_on_drive = os.listdir(drive_folder_path)
  files_in_localdir = os.listdir(local_folder_path)
  files_to_download = [f for f in files_on_drive if f not in files_in_localdir]
  files = [(filename, get_id(os.path.join(drive_folder_path, filename))) for filename in files_to_download]

  async with aiohttp.ClientSession() as session:
    tasks = []
    for idx, (name, file_id) in enumerate(files):
      local_file_path = os.path.join(local_folder_path, name)
      task = asyncio.create_task(download_file(session, name, file_id, local_file_path, token))
      tasks.append(task)

    # 使用 tqdm 包装任务列表以显示进度条
    for f in tqdm.as_completed(tasks, total=len(tasks)):
      await f




def process_data2(traindir, shift=0, termi=10, dropFalse=0.85, dropPos=0)\
 -> tuple[torch.tensor, torch.tensor]:
  """Reading data in "traindir" and process them.
  Taking care of memory efficiency"

  Args:
    shift: 從第幾個開始讀取
    termi: 讀取幾個
    dropFalse: 被刪除的有闌尾炎的張數比例
    dropPos: 被刪除的有闌尾炎的張數比例

  Returns:
    images: 输出的图像数据
    labels: 输出的标签数据
  """
  print(f"reading {traindir=}...")
  labels_ = read_label(labelpath)

  if labels_.index.name != 'id':
    labels_.set_index('id', inplace=True)

  #裁減過的圖片放在Cropped_[範圍]的資料夾下,用路徑名稱判斷資料使否裁減過
  cropmatch = re.search(r'Cropped', traindir)
  if not cropmatch:
    print("讀取未切片的資料夾")
    xlim, ylim, zlim = [0,512], [0,512], [0,None]
  else:
    #範圍 = xstart-xend_ystart-yend_zstart-zend
    ismatch = re.search(r'(\d+-\d+)_(\d+-\d+)_(\d+-\d+)', traindir)
    if ismatch:
      dimens = ismatch.group(0)
      labelidx = dimens.split('_')
      xlim = [int(idx) for idx in labelidx[0].split('-')]
      ylim = [int(idx) for idx in labelidx[1].split('-')]
      zlim = [int(idx) for idx in labelidx[2].split('-')]
      print(f"{xlim=}\n{ylim=}\n{zlim=}")
      if xlim[0] >= xlim[1] or ylim[0] >= ylim[1] or zlim[0] >= zlim[1]:
        print("切片格式不正确")
        return
    else:
      print("切片格式不正确")
      return


  filelist = os.listdir(traindir)
  filelist = filelist[shift:shift+termi]

  # preallocated
  if not cropmatch:
    selecteds = [labels_.loc[labels_.index.str.startswith(afile.strip('.nii.gz')+'_')] for afile in filelist]
    numcuts = [len(selected) for selected in selecteds]
    numcuts = sum(numcuts)
  else:
    numcuts = (zlim[1]-zlim[0]) * len(filelist)
  images = torch.zeros(numcuts,1, ylim[-1]-ylim[0],xlim[-1]-xlim[0])
  labels = -torch.ones(numcuts)
  nimgs = 0
  nprocess = 0

  for key in filelist:
    file_path = os.path.join(traindir, key)
    key = key.strip('.nii.gz')
    scan  = labels_.loc[labels_.index.str.startswith(key+'_')]
    value = nib.load(file_path).get_fdata()

    label_t = torch.tensor(scan['label'][zlim[0]:zlim[-1]])
    image_t = torch.from_numpy(value).float().permute(2, 0, 1).unsqueeze(1)

    #有幾筆不是512x512
    if (not cropmatch) and (image_t.shape[2] != 512 or image_t.shape[3] != 512):
      image_t = cropping(image_t)

    image_t, label_t = remove_false_images(image_t, label_t, dropFalse)
    image_t, label_t = remove_positive_images(image_t, label_t, dropPos)

    n_new = len(label_t)

    images[nimgs:nimgs+n_new] = image_t
    labels[nimgs:nimgs+n_new] = label_t
    nimgs += len(label_t)
    print(f"Process {nprocess}: {key} finished...")
    nprocess += 1


  #after drop
  counts = count_zero(images)
  images = torch.asarray(images[:len(images)-counts])
  labels = labels[labels != -1]

  return images, labels.float()




def read_test(testdir,shift=0,termi=10):
  """Reading data in "testdir"
  not memory efficient
  Args:
    shift: 從第幾個開始讀取
    termi: 讀取幾個
  Returns:
    images: 输出的图像数据
    scans_info: 每個scan的檔名和切片數(原本nii檔所包含的數量)

  """


  filelist = os.listdir(testdir)
  filelist = filelist[shift:shift+termi]
  scans_info = []

  images = []
  nprocess = 0

  for key in filelist:
    file_path = os.path.join(testdir, key)
    key = key.strip('.nii.gz')
    value = nib.load(file_path).get_fdata()
    scans_info.append((key, value.shape[2]))

    image_t = torch.from_numpy(value).float().permute(2, 0, 1).unsqueeze(1)

    images.append(image_t)
    print(f"{nprocess}: Read test {key} finished...")
    nprocess += 1

  images = torch.cat(images,dim=0)
  print(f"read {len(scans_info)} scans")

  return images, scans_info




def save_model(model, modelname="", dirname="", root=os.path.realpath("/content/drive/MyDrive/AOCR2024/params")):

  if not model:
    print("not given model")
    return

  if not os.path.exists(root):
    print(f"{root} not exists!")
    return

  print(f"model will be saved as {root}/{dirname}/{modelname}")

  if not modelname:
    modelname = input("請輸入模型儲存的檔名:")
  if not dirname:
    dirname = input("請輸入模型儲存的資料夾:")

  filename = f"{root}/{dirname}/{modelname}"

  if not os.path.exists(os.path.dirname(filename)):
    os.mkdir(os.path.dirname(filename))

  if os.path.isfile(filename+'.pth'):
      print(f"{filename}.pth exist.")
  else:
      torch.save(model.state_dict(), f'{filename}.pth')

  return filename



def read_label(excel_path) -> pd.DataFrame:
  """Reads a csv file containing ground-truth.
    The csv file should have two columns: 'id' and 'label'.
  """
  with open(excel_path, 'r') as f:
    df = pd.read_csv(f)
    df.set_index('id', inplace=True)
    return df



def write_error(logpath, e, nprocess):
  # with open(logpath, 'a') as f:
    # f.write(f"{nprocess:} error occured: {e}\n") ##! may stuck the program!!
  print(f"{nprocess:} error occured: {e}\n")


def read_submission(excel_path) -> pd.DataFrame:
  """Reads a csv file containing submission file.
    The csv file should follow the format given by Kaggle.
  """
  with open(excel_path, 'r') as f:
    df = pd.read_csv(f)
    df.set_index('id', inplace=True)
    return df




## 列印結果

In [ ]:
def get_confusion(guess, truth):
  right = guess == truth
  wrong = np.logical_not(right)
  TP = np.sum(np.logical_and(right, truth == np.ones(right.shape)))
  TN = np.sum(np.logical_and(right, truth == np.zeros(right.shape)))
  FN = np.sum(np.logical_and(wrong, truth == np.ones(wrong.shape)))
  FP = np.sum(np.logical_and(wrong, truth == np.zeros(wrong.shape)))
  return (TP,FP,FN,TN)



def print_results(prediction, labels):
  if torch.is_tensor(prediction):
    prediction = prediction.cpu().numpy()
  if torch.is_tensor(labels):
    labels = labels.cpu().numpy()

  TP,FP,FN,TN = get_confusion(prediction, labels)
  print(f"\n\
      真實值\n\
  預  +-----+-----+\n\
  測| TP: {TP}| FP: {FP}|\n\
  值| FN: {FN}| TN: {TN}|\n\
      +-----+-----+ \n")

  recall = TP/(TP + FN)
  precision = TP/(TP + FP)
  recall = 0 if np.isnan(recall) else recall.item()
  precision = 0 if np.isnan(precision) else precision.item()
  F1 = 0 if recall + precision == 0 else  (2*recall*precision/(recall+precision))

  print(f"{recall=}\n{precision=}\n{F1=}\n")

## 數據裁減




In [ ]:
def argumenting(images, labels, n=5):
  """
  數據增強。
  在images裡面把所有有闌尾炎的cut複製給定次數，並
  插入回images的隨機位置裡面

  Args:
    images: 输入的图像数据
    labels: 输入的标签数据
    n: 每个样本複製多少次

  Returns:
    增強后的图像数据和标签数据
  """
  where = (labels == True).nonzero(as_tuple=True)[0]
  bad_images = images[where]

  rep_imgs = bad_images.repeat(n, 1, 1, 1)
  rep_labels = torch.ones(rep_imgs.shape[0], dtype=labels.dtype)

  # import pdb
  # pdb.set_trace()
  # 隨機插入argumented圖片
  nimg = images.shape[0]
  rnd_pos = torch.randint(0, nimg, (rep_imgs.shape[0],))
  images = torch.cat((images, rep_imgs), dim=0)
  images = images[torch.argsort(torch.cat((torch.arange(nimg), rnd_pos)))]

  # 對應對置插入標籤
  labels = torch.cat((labels, rep_labels), dim=0)
  labels = labels[torch.argsort(torch.cat((torch.arange(nimg), rnd_pos)))]

  return images, labels

def remove_false_images(images, labels, ratio):
    """
    隨機在images裡面移除一定比例的無闌尾炎cut

    Args:
      images: 输入的图像数据
      labels: 输入的标签数据
      ratio: 移除的比例

    Returns:
      移除后的图像数据和标签数据
    """
    # 找出 labels == 0 的索引
    where_false = (labels == 0).nonzero(as_tuple=True)[0]
    mask = torch.ones(len(images), dtype=torch.bool)
    indices_filter = torch.randperm(len(where_false))[:int(len(where_false)*ratio)]
    mask[where_false[indices_filter]] = False
    images = images[mask]
    labels = labels[mask]

    return images, labels

def remove_positive_images(images, labels, ratio):
    # 找出 labels == 1 的索引
    where_positive = (labels == 1).nonzero(as_tuple=True)[0]
    mask = torch.ones(len(images), dtype=torch.bool)
    indices_filter = torch.randperm(len(where_positive))[:int(len(where_positive)*ratio)]
    mask[where_positive[indices_filter]] = False
    images = images[mask]
    labels = labels[mask]

    return images, labels

## 其他

In [ ]:

def count_zero(images):
  count = 0
  for image in images:
    if not torch.any(image):
      count += 1
  return count

def cropping(image):
  if image.shape[2] != 512:
    start = (image.shape[2] - 512) // 2
    end = start + 512
    image = image[:, :, start:end, :]

  if image.shape[3] != 512:
    start = (image.shape[3] - 512) // 2
    end = start + 512
    image = image[:, :, :, start:end]
  return image




def custom_sort_key(val):
    parts = val.split('_')
    if len(parts) == 2 and parts[1].isdigit():
        return (parts[0], int(parts[1]))
    return (parts[0], -1)  # 使沒有_(數字)的id排最前面


def isgpu():
    """檢查是否有 CUDA 支持的 GPU"""
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("GPU is available")
    else:
        device = torch.device("cpu")
        # raise("GPU not available")
    return device


In [ ]:
device = isgpu()

# 掛載


In [ ]:
from google.colab import drive
drive.mount('/content/drive/',)

In [ ]:
!pwd

In [ ]:
drive_rootdir = "/content/drive/MyDrive/AOCR2024"
labelpath = '/content/drive/MyDrive/AOCR2024/TrainValid_ground_truth.csv'
# datadir ="Train_Valid_Image"
datadir ="Cropped_60-316_150-406_11-71/Train_Valid_Image_cropped"


Use service account to access googledrive

In [ ]:
import io
import googleapiclient.discovery
from google.auth.exceptions import TransportError
from google.oauth2.service_account import Credentials
from googleapiclient.http import MediaIoBaseDownload

DriveAPIcredentials = r"/content/drive/MyDrive/AOCR2024/creds/appendicitis-407217-c2f141040794.json"
credentials = Credentials.from_service_account_file(DriveAPIcredentials,scopes=["https://www.googleapis.com/auth/drive"])
drive_service = googleapiclient.discovery.build('drive', 'v3', credentials=credentials)

# 資料下載(非同步)

In [ ]:
from google.auth.transport.requests import Request
request = Request()
credentials.refresh(request)
loop.run_until_complete(get_data_disk_async(drive_rootdir, datadir,token = credentials.token))

Use http request pulic links of google drive files  to download (error 403. be blocked from google)

In [ ]:
!curl ipecho.net/plain

In [ ]:
len(os.listdir(datadir))

Directly reading files from mounted google drive  
**unknown error**: stuck after reading 9 nii files

In [ ]:
# # 測試直接從mount下載
# labelpath = '/content/drive/MyDrive/AOCR2024/TrainValid_ground_truth.csv'
# datadir ="/content/drive/MyDrive/AOCR2024/Train_Valid_Image"

# process_data2(termi=5,shift=0,dropFalse=0,dropPos=0)

In [ ]:
# # 測試下載
# labelpath = '/content/drive/MyDrive/AOCR2024/TrainValid_ground_truth.csv'

# dflabel = read_label(labelpath)
# images, labels, _ =  process_data2(termi=3,shift=2,dropFalse=0,dropPos=0)

In [ ]:
# s1 = labels[0:60]
# s2 = labels[60:120]
# s3 = labels[120:180]
# s2

# Dataset/Model

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, mode='train'):
        self.images = images
        self.labels = labels
        self.mode = mode

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
      if self.mode == 'train' or self.mode=='valid':
        return self.images[idx], self.labels[idx]
      elif self.mode == 'test':
        return self.images[idx]

In [ ]:
model_name = "tf_efficientnetv2_m"
pretrained_model = timm.create_model(model_name, pretrained=True)

# 修改輸入通道
pretrained_model.conv_stem = nn.Conv2d(1, 24, kernel_size=3, stride=2, padding=1, bias=False)

# 修改輸出類別
num_classes = 1
pretrained_model.classifier = nn.Linear(pretrained_model.classifier.in_features, num_classes)

# 添加 Sigmoid 激活函數
pretrained_model = nn.Sequential(
    pretrained_model,
    nn.Sigmoid()
)


# 檢查模型結構
# print(pretrained_model)


In [ ]:
def train(loader, model_, optimizer, criterion, device):
    model_.train()
    running_loss = 0.0
    correct_count = 0
    total_count = 0

    for batch_images, batch_labels in loader:
        batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)
        optimizer.zero_grad()
        outputs = model_(batch_images).squeeze()
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        predicted = (outputs > 0.5).int()
        correct_count += (predicted == batch_labels).sum().item()
        total_count += batch_labels.size(0)

    epoch_loss = running_loss / len(loader)
    epoch_accuracy = correct_count / total_count

    return epoch_loss, epoch_accuracy


def valid(loader, model_, criterion, device):
  model_.eval()
  testing_loss = 0.0
  correct = 0
  total = 0
  predict_list = torch.tensor([]).to(device)

  with torch.no_grad():
      for batch_images, batch_labels in loader:
          batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)
          outputs = model_(batch_images).squeeze()
          predicted = (outputs > 0.5).int()
          loss = criterion(outputs, batch_labels)

          testing_loss += loss.item()
          total += batch_labels.size(0)
          correct += (predicted == batch_labels).sum().item()
          predict_list = torch.cat((predict_list, predicted), 0)

  valid_loss = testing_loss / len(loader)
  valid_accuracy = 100 * correct / total

  return valid_loss, valid_accuracy, predict_list




# 資料處理

In [ ]:
dflabel = read_label(labelpath)
total_nii = len(os.listdir(datadir))
images, labels =  process_data2(traindir=datadir, termi=10,shift=0,dropFalse=0,dropPos=0)

In [ ]:
# images_train, labels_train = remove_false_images(images, labels, 0.9)
# print(images_train.shape)
# print(len(labels_train))
# print(images.shape)

In [ ]:
# len(labels_train)

In [ ]:
# labels_train

In [ ]:
#  %reset_selective images_train, labels_train

In [ ]:
# %reset_selective dataset

In [ ]:
# %reset_selective dataloader

In [ ]:
# %reset

# 訓練 (EfficiencyNetV2_m)

第一次跑model 跑這


In [ ]:
model = pretrained_model
model = model.to(device)
start = 0

若是有接續跑模型   
跑這epoch要對上

In [ ]:
model = pretrained_model
model = model.to(device)

import json
filename = input("請輸入要獲取模型路徑:")

if not os.path.isfile(filename+'.pth'):
    print(f"{filename}.pth not exist.")
else:
    print(model.load_state_dict(torch.load(filename+'.pth')))

if os.path.isfile(filename+'.json'):
  with open(f'{filename}.json', 'r') as f:
      params = json.load(f)

start = int(filename.split('_')[-1]) + 1

In [ ]:
num_epochs = 190
batch_size = 16
lr = 0.01
running_loss_list = []
modeldirname = input("dirname: ")
droprate = 0.95

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr)

images_train, labels_train = remove_false_images(images, labels, droprate)
dataset = CustomDataset(images_train, labels_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)



for epoch in range(start, num_epochs):

    epoch_loss, epoch_accuracy = train(dataloader, model, optimizer, criterion, device)
    running_loss_list.append(epoch_loss)
    print(f"Epoch {epoch}, Loss: {epoch_loss}, Accuracy: {epoch_accuracy}")

    images_train, labels_train = remove_false_images(images, labels, droprate)
    dataset = CustomDataset(images_train, labels_train)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

    if epoch % 10 == 0:
      savepath = save_model(model, f'{model_name}_{epoch}', dirname=modeldirname)

savepath = save_model(model, f'{model_name}_{num_epochs}', dirname=modeldirname)


## Loss圖

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(1,num_epochs+1)[:len(running_loss_list)], running_loss_list)
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

# figname = f'{os.path.dirname(savepath)}/{model_name}_loss.png'
# if os.path.isfile(figname):
#   print(f"{figname} exist.")
# else:
#   plt.savefig(figname)
plt.show()

lossname = f'{os.path.dirname(savepath)}/{model_name}_loss.json'

with open(f'{lossname}', 'a') as f:
  json.dump(running_loss_list, f)

# 儲存模型參數



In [ ]:
filename = save_model(model)

params = {
    'num_epochs': num_epochs,
    'batch_size': batch_size,
    'learning_rate': lr,
}

if os.path.isfile(filename+'.json'):
    print(f"{filename}.json exist.")
else:
    with open(f'{filename}.json', 'w') as f:
        json.dump(params, f)

# 讀取模型參數

In [ ]:
model = pretrained_model
model = model.to(device)

In [ ]:
import json
filename = input("請輸入要獲取模型路徑:")

if not os.path.isfile(filename+'.pth'):
    print(f"{filename}.pth not exist.")
else:
    print(model.load_state_dict(torch.load(filename+'.pth')))

if os.path.isfile(filename+'.json'):
  with open(f'{filename}.json', 'r') as f:
      params = json.load(f)

# 評估


In [ ]:
# 評估設置
try:
  batch_size = params['batch_size']
except:
  batch_size = int(input("batch_size:"))
print(batch_size)

dataset = CustomDataset(images_train, labels_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
criterion = nn.BCELoss()
valid_loss, valid_accuracy, predict_list = valid(dataloader, model, criterion, device)
print(f"Accuracy of the network on the test images: {valid_accuracy}%")
print(f"Loss: {valid_loss}")
predict_listq = predict_list.cpu()


Accuracy of the network on the 15592   
training images: 99.20472036942022%  
Loss: 0.03187363331013983


In [ ]:
print_results(predict_listq, labels)

# Test

images讀進來是疊在一起的。 需要info紀錄images檔名順序與分割情況

In [ ]:
testdir = "Cropped_60-316_150-406_11-71/Test_Image_cropped"

In [ ]:
from google.auth.transport.requests import Request
request = Request()
credentials.refresh(request)
loop.run_until_complete(get_data_disk_async(drive_rootdir, testdir,token = credentials.token))

In [ ]:
images_test, info = read_test(testdir,termi=200,shift=0)

In [ ]:
images_test.shape

In [ ]:
%reset_selective images_test

In [ ]:
try:
  batch_size = params['batch_size']
except:
  batch_size = int(input("batch_size:"))
print(batch_size)

dataset = CustomDataset(images_test, None, mode='test')
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


In [ ]:
model.eval()

predict_list = torch.tensor([]).to(device)
with torch.no_grad():  # 不更新梯度
    for j, batch_images in enumerate(dataloader):
        batch_images = batch_images.to(device)
        outputs = model(batch_images)
        predicted = (outputs.squeeze() > 0.5).int()

        # if predicted.dim() == 0:
        #   predicted = predicted.unsqueeze(0)
        predict_list = torch.cat((predict_list,predicted),0)


predict_listq = predict_list.cpu()

In [ ]:
predict_listq.sum()

# 輸出至submission.csv

訓練過程不改變資料順序, 所以輸出順序同輸入.   
predict值跟隨images順序, 照著info把每個predict值(label)與檔名(id)對上. 輸出成提交格式

In [ ]:
def cont(l):
  "1是否連續"
  f1 = False
  f2 = False
  for i in range(len(l)):
    if (l[i] == 1):
      f1 = True
    if (l[i] == 0):
      if (f1):
        f2 = True
    if (l[i] == 1) and f2:
      return False
  return True

def yes(predict):
  return int(sum(predict) >= 3)



predict_list = predict_listq.numpy()

## 還原id與對應label，假設key按照scan輸入順序排列,每個key對應的scan的cuts數是nslice
output = {}
k = 0  #第幾個scan
ii = 0   #每個key輪到第幾個
id, nslice = info[k][0], info[k][1]
for i in range(len(predict_list)):

  if (ii >= nslice):
    #該換下一個scan了
    output[id] = yes(predict_list[i-nslice:i]) #評估方式

    k += 1
    ii = 0
    id, nslice = info[k][0], info[k][1]

  label = predict_list[i]
  output[id+f'_{ii}'] = int(predict_list[i])
  ii += 1

output[id] = yes(predict_list[(i+1)-ii:]) #補上最後一個scan評估
# import pdb
# pdb.set_trace()
output = list(output.items())
dfout = pd.DataFrame(output)
dfout.columns = ['id', 'label']
dfout = dfout.sort_values(by='id', key=lambda x: x.map(custom_sort_key))
filename = input("輸入提交路徑(default for submission)")
if filename == '':
  filename = 'submission'
dfout.to_csv(f"{filename}"+'.csv', index=False)

In [ ]:
info

In [ ]:
dfout

# 查看submission.csv

In [ ]:
!pwd

In [ ]:
dftest = read_submission('fisrt_80.csv')
# dftest = read_submission('submission.csv')
dflabel = read_label("TrainValid_ground_truth.csv")

In [ ]:
len(dftest)

In [ ]:
import re


# 抓出scan-level data
pattern = re.compile(r'.*_[0-9]+$')  # 正則表達式匹配 "_數字" 結尾
mask = ~dftest.index.str.match(pattern)
scan_guess = np.array(dftest[mask]['label'])
scan_truth = np.array(dflabel.loc[dftest[mask]['label'].index]['label'])

mask = ~mask
cut_guess = np.array(dftest[mask]['label'])
cut_truth = np.array(dflabel.loc[dftest[mask]['label'].index]['label'])

In [ ]:
ss = 10
print("="*ss + "F1 score on scan level" + "="*ss)
print_results(scan_guess, scan_truth)
print(),print()
print("="*ss + "F1 score on cut level" +"="*ss )
print_results(cut_guess, cut_truth)

# 其他指令
不在工作流
當參考

In [ ]:
import time
datadir = 'Train_Valid_Image'
for idx, afile in enumerate(os.listdir(datadir)):
  file_path = os.path.join(datadir, afile)
  time.sleep(1)
  nii_file =  nib.load(file_path)
  print(f"{idx}: read {afile}")

In [ ]:
pattern = re.compile(r'.*_[0-9]+$')  # 正則表達式匹配 "_數字" 結尾
mask = dftest.index.to_series().str.match(pattern)
dflabel_ = dftest[mask]
dflabel_.index.map(lambda x : x.split('_')[1]).sort_values()[-3:]

In [ ]:
import random

# 假設這是您的列表
my_list = np.array([1, 2, 3, 4])

mask = [True, False, False, True]
my_list[mask]

In [ ]:
datadir ="TrainValid_Image/train_data"
labelpath = 'TrainValid_ground_truth.csv'
labels_ = read_label(labelpath)


filelist = os.listdir(datadir)
filelist = filelist[10:20]
selecteds = [labels_.loc[labels_.index.str.startswith(afile.strip('.nii')+'_')] for afile in filelist]
scans_info = []
# preallocated
print("enter")
numcuts = [len(selected) for selected in selecteds]
numcut = sum(numcuts)

In [ ]:
numcuts = [len(selected) for selected in selecteds]

In [ ]:
img = nib.load('Train_Valid_Image/Zx00AD16F8B97A53DE6E7CFE260BDF122F0E655659A3DF1628.nii.gz')


In [ ]:
drive.flush_and_unmount()

In [ ]:
t1 = torch.randint(3,(5,1,2,2))
lab = torch.tensor([1,1,1,1,1])
t1

In [ ]:
remove_positive_images(t1,lab,0.79)

In [ ]:
!pip install -U --no-cache-dir gdown gdown

In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
!gdown https://drive.google.com/drive/folders/1C7HXpHMw1Alvwif9hO97FUzfn4rhxG8B -O Train_Valid_Image --folder --remaining-ok